# Manual CPA Attack

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

In [16]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()

In [17]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'TINYAES128C'
proj_path = "projects/M2_CPA"
num_traces = 100
CHECK_CORR = False

## The CPA Attack Theory

As a background on the CPA attack, please see the section [Correlation Power Analysis](https://wiki.newae.com/Correlation_Power_Analysis). It's also suggested that you complete Tutorial B1, since that will introduce Jupyter and show how to interface with ChipWhisperer using Python. It's assumed you've read that section and completed the tutorial and come back to this. Ok, you've done that? Good let's continue.

Assuming you **actually** read that, it should be apparent that there is a few things we need to accomplish:

1. Getting some power traces of our target while it's performing AES encryption.
1. Reading the data, which consists of the analog waveform (trace) and input text sent to the encryption core
1. Making the power leakage model, where it takes a known input text along with a guess of the key byte
1. Implementing the correlation equation, and then looping through all the traces
1. Ranking the output of the correlation equation to determine the most likely key.

This tutorial will deal with both recording power traces using ChipWhisperer and breaking them using a CPA attack.

## Capturing Power Traces

Capturing power traces will be very similar to previous tutorials, except this time we'll be using a loop to capture multiple traces, as well as numpy to store them. It's not necessary, but we'll also plot the trace we get using `bokeh`.

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [18]:
HEX_PATH =  "./fu262.hex"

In [19]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, HEX_PATH)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 17503 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 17503 bytes


### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and appends our new trace to the `traces[]` list. At the end, we convert the trace data to numpy arrays, since that's what we'll be using for analysis.

In [20]:
target = cw.target(scope, noflush=True) # MUST HAVE noflush=True
print(target.baud) # make sure this baud rate is set in your project

38400


In [21]:
PROTOCOL_NOP = 0x10
PROTOCOL_XOR = 0x20
PROTOCOL_DES = 0x30
PROTOCOL_AES = 0x40

MAGIC_BYTE = 0xFE

RESPONSE_SUCCESS = 0x00
RESPONSE_FAILURE = 0x01
RESPONSE_UNKNOWN_PROTOCOL = 0x02

def response_code_to_string(code):
    code = ord(code)
    if code == RESPONSE_SUCCESS:
        return "SUCCESS"
    elif code == RESPONSE_FAILURE:
        return "FAILURE"
    elif code == RESPONSE_UNKNOWN_PROTOCOL:
        return "UNKNOWN_PROTOCOL"
    else:
        return "UNKNOWN_RESPONSE_CODE"

# reads n bytes from target uart
def read_uart(target, n_bytes):
    response = []
    while len(response) < n_bytes:
        response += target.read()
    return response

In [22]:
# #convert the message to a list of bytes
from time import sleep
from Crypto.Cipher import DES
from Crypto.Cipher import AES
message = "I am the senate"

message = [ord(c) for c in message]
#put the magic byte at the beginning of the message
message_plaintext = [MAGIC_BYTE] + message
#message_plaintext = message

sleep(0.1)

print("===============MESSAGE_AES=================")

#make a copy of the message to send with the aes protocol
message_aes = message_plaintext.copy()

#we're using AES-128 and the message is 16 bytes long

#prepend the protocol byte
message_aes = bytes([PROTOCOL_AES]) + bytes(message_aes)

#print the hex representation of the message_aes
print("Message_aes :" + " ".join("{:02x}".format(c) for c in message_aes))

# start scoping before sending decrypt request
scope.arm()

#write the message to the uart
target.write(bytes(message_aes))

# read captured trace
ret = scope.capture()
trace_aes = scope.get_last_trace()

#read the 17 byte response
response_aes = read_uart(target, 17)

#print the hex representation of the response
print("Response_aes:" + " ".join("{:02x}".format(ord(c)) for c in response_aes))

#check the response code
print("Response code: {}".format(response_code_to_string(response_aes[0])))

#convert the response_aes to a printable string (excluding the response code and magic byte)
response_aes = "".join(response_aes[2:])
print("Response_aes: {}".format(response_aes))

sleep(0.1)

===============MESSAGE_AES=================
Message_aes :40 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response_aes:01 55 38 8c a9 a9 45 05 04 cc f4 f3 4d 4a ed 48 f3
Response code: FAILURE
Response_aes: 8©©EÌôóMJíHó


In [23]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []
textin_array = []

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    textin_array.append(text)
    
    #make a copy of the message to send with the aes protocol
    #message_aes = message_plaintext.copy()
    message_aes = text

    #we're using AES-128 and the message is 16 bytes long

    #prepend the protocol byte
    message_aes = bytes([PROTOCOL_AES]) + bytes(message_aes)

    #print the hex representation of the message_aes
    print("Message_aes :" + " ".join("{:02x}".format(c) for c in message_aes))

    # start scoping before sending decrypt request
    scope.arm()

    #write the message to the uart
    target.write(bytes(message_aes))

    # read captured trace
    ret = scope.capture()
    trace = scope.get_last_trace()
    
    if trace is None:
        continue
    traces.append(trace)

    #read the 17 byte response
    response_aes = read_uart(target, 17)

    #print the hex representation of the response
    print("Response_aes:" + " ".join("{:02x}".format(ord(c)) for c in response_aes))

    #check the response code
    print("Response code: {}".format(response_code_to_string(response_aes[0])))

    #convert the response_aes to a printable string (excluding the response code and magic byte)
    response_aes = "".join(response_aes[2:])
    print("Response_aes: {}".format(response_aes))

    sleep(0.1)


#Convert traces to numpy arrays
trace_array = np.asarray([trace for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin_array)
#known_keys = np.asarray([trace.key for trace in traces])  # for fixed key, these keys are all the same

C:\Users\sghaz\AppData\Local\Temp\ipykernel_20004\13059178.py:11: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(num_traces, desc='Capturing traces'):


Capturing traces:   0%|          | 0/100 [00:00<?, ?it/s]

Message_aes :40 d3 b0 ff a2 66 d6 ae 33 75 76 e8 a9 ce 45 ef 80
Response_aes:01 39 61 4b 6d 7a d5 01 85 88 a9 e8 6f 7d 6c 6f d1
Response code: FAILURE
Response_aes: aKmzÕ©èo}loÑ
Message_aes :40 79 61 c2 8b 31 58 e5 a3 ee 48 21 06 0e ce 1a 8a
Response_aes:01 50 bf 45 c5 e8 35 4c 6f 59 2d c8 9e ae f7 49 61
Response code: FAILURE
Response_aes: ¿EÅè5LoY-È®÷Ia
Message_aes :40 10 ec 63 02 1c 49 3d 59 2b 56 28 0d 21 e9 14 f2
Response_aes:01 57 ca f4 47 2d df 2c e3 db a6 77 e4 8a f1 97 53
Response code: FAILURE
Response_aes: ÊôG-ß,ãÛ¦wäñS
Message_aes :40 c8 ea a0 53 4f 62 ce 52 23 c0 7e f6 bc 7f 3c 4f
Response_aes:01 7c 58 3f 47 19 dd 08 b5 d4 e6 fe f5 11 56 36 a9
Response code: FAILURE
Response_aes: X?GµÔæþõV6©
Message_aes :40 3e 17 b1 79 5f 00 28 75 c7 6f 80 33 a0 c0 24 fa
Response_aes:01 44 43 77 e0 b9 b7 70 47 72 f9 32 fe fa 8a 3e eb
Response code: FAILURE
Response_aes: Cwà¹·pGrù2þú>ë
Message_aes :40 8a 8e 09 50 5f d9 dd 4f a2 f5 f7 00 03 9b 61 63
Response_aes:01 13 c8 4a 53 a0 74

Message_aes :40 e3 1d f9 a3 83 a0 b8 09 d3 1f 2e 4f 38 55 19 ad
Response_aes:01 fc 4c 5d cc 09 9f 25 de 24 e5 4b ec 0a f7 17 1e
Response code: FAILURE
Response_aes: L]Ì	%Þ$åKì
÷
Message_aes :40 03 34 e8 a3 45 49 4f 01 76 c2 b4 df c9 2e 61 b9
Response_aes:01 fa 7f e6 36 30 12 12 c6 9f c4 d3 83 28 e5 74 7c
Response code: FAILURE
Response_aes: æ60ÆÄÓ(åt|
Message_aes :40 cc f2 32 02 1a 52 64 30 60 51 be 9e d6 34 37 c3
Response_aes:01 57 35 9f b0 c3 31 e6 fa 1d b8 d7 fd 66 e2 cd 13
Response code: FAILURE
Response_aes: 5°Ã1æú¸×ýfâÍ
Message_aes :40 d3 47 80 d6 d7 22 f5 6a 7c cc 1c b0 4a 00 97 7d
Response_aes:01 6d e4 7f 1b e0 f3 3c 8b f1 51 11 1a 4d 4f 0a 31
Response code: FAILURE
Response_aes: äàó<ñQMO
1
Message_aes :40 19 02 ae 85 42 d2 97 c2 3c 09 56 3b b9 93 1b 2f
Response_aes:01 7a 37 1a e5 79 27 cb de 78 eb fa 25 1d af 6c 1f
Response code: FAILURE
Response_aes: 7åy'ËÞxëú%¯l
Message_aes :40 ca 2f bb 06 13 54 48 d0 e4 d5 30 05 4c 89 47 7c
Response_aes:01 60 bc a6 db 37 

Message_aes :40 20 10 20 e0 11 c1 fb df 20 59 b3 71 9c e3 b6 1b
Response_aes:01 a9 cd a6 d0 02 ad a0 a8 18 89 66 85 bb bd 2b 91
Response code: FAILURE
Response_aes: Í¦Ð­ ¨f»½+
Message_aes :40 3d a3 9b 97 6c e9 64 1a 30 72 4f 88 14 07 1f 90
Response_aes:01 89 41 80 5a b8 54 fb 69 8a 0e 2f b6 15 49 3e 19
Response code: FAILURE
Response_aes: AZ¸Tûi/¶I>
Message_aes :40 14 25 9c 29 84 9b 36 7a d4 a0 82 6b 91 20 4c 9f
Response_aes:01 15 02 38 a1 44 16 d6 e3 d1 03 2a a1 1a 8f 37 6a
Response code: FAILURE
Response_aes: 8¡DÖãÑ*¡7j
Message_aes :40 97 29 c7 dd 08 ae dc 84 83 02 15 2f 0a 8f 90 88
Response_aes:01 75 e2 ec 66 38 12 cf b3 e6 c5 44 df 4e 2a 6a f7
Response code: FAILURE
Response_aes: âìf8Ï³æÅDßN*j÷
Message_aes :40 31 cb 47 6b 01 b4 36 8f 53 1d e8 66 3f 39 4c c7
Response_aes:01 2f 53 54 7f ab 6a 3a 27 b9 25 7f b8 0a 93 26 5c
Response code: FAILURE
Response_aes: ST«j:'¹%¸
&\
Message_aes :40 5a 28 d0 72 ce 24 7f c7 2a 15 53 c7 2e c5 42 ca
Response_aes:01 f9 4e 11 82 93 

Now that we have our traces, we can also plot them using Bokeh:

In [24]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(traces[0]))
p.line(xrange, traces[2], line_color="red")
show(p)

Loading BokehJS ...

In [25]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Trace Analysis

### Using the Trace Data

Now that we have some traces, let's look at what we've actually recorded. Looking at the earlier parts of the script, we can see that the trace data is in `trace_array`, while `textin_array` stores what we sent to our target to be encrypted. For now, let's get some basic information (the total number of traces, as well as the number of sample points in each trace) about the traces, since we'll need that later:

In [26]:
numtraces = np.shape(trace_array)[0] #total number of traces
#print(numtraces)
numpoints = np.shape(trace_array)[1] #samples per trace
#print(numpoints)

For the analysis, we'll need to loop over every byte in the key we want to attack, as well as every trace:
```python
for bnum in range(0, 16):
    for tnum in range(0, numtraces):
        pass
```
Though we didn't loop over them, note that each trace is made up of a bunch of sample points.
Let's take a closer look at AES so that we can replace that `pass` with some actual code.

### Calculating Correlation

Now that we have some power traces of our target that we can use, we can move on to the next steps of our attack. Looking way back to how AES works, remember we are effectively attemping to target the position at the bottom of this figure:

![title](https://wiki.newae.com/images/7/71/Sbox_cpa_detail.png)

The objective is thus to determine the output of the S-Box, where the S-Box is defined as follows:

In [27]:
sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

Thus we need to write a function taking a single byte of input, a single byte of the guessed key, and return the output of the S-Box:

In [28]:
def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

Finally, remember we want the Hamming Weight of the guess. Our assumption is that the system is leaking the Hamming Weight of the output of that S-Box. As a dumb solution, we could first convert every number to binary and count the 1's:

```python
>>> bin(0x1F)
'0b11111'
>>> bin(0x1F).count('1')
5
```
This will ultimately be fairly slow. Instead we make a lookup table using this idea:

In [29]:
HW = [bin(n).count("1") for n in range(0, 256)]

### Performing the Check

Remember the objective is to calculate the following:
$$r_{i,j} = \frac{\sum_{d=1}^{D}[(h_{d,i} - \bar{h_i})(t_{d,j}-\bar{t_j})]}{\sqrt{\sum_{d=1}^D(h_{d,i}-\bar{h_i})^2\sum_{d=1}^D(t_{d,j}-\bar{t_j})^2}}$$

Where

| **Equation** | **Python Variable** | **Value**  | 
|--------------|---------------------|------------|
|  d           |       tnum          | trace number |
|  i           |       kguess        | subkey guess |
| j | j index trace point | sample point in trace |
| h | hypint | guess for power consumption | 
| t | traces | traces | 

It can be noticed there is effectively three sums, all sums are done over all traces. For this initial implementation we'll be explicitly calculating some of these sums, although it's faster to use NumPy to calculate across large arrays. We'll convert those three summations into variables, turning the equation into this format:

$$r_{i,j}=\frac{sumnum}{\sqrt{snumden1 * sumden2}}$$

Where:

$$sumnum = \sum_{d=1}^{D}[(h_{d,i} - \bar{h_i})(t_{d,j}-\bar{t_j})]$$

$$sumden1 = \sum_{d=1}^D(h_{d,i}-\bar{h_i})^2$$

$$sumden2 = \sum_{d=1}^D(t_{d,j}-\bar{t_j})^2$$

Looking at this, we can see that we'll need $\bar{h_i}$ and $\bar{t_j}$, so let's start by building some code that will give us those. Looking at the CPA tutorial, we can see that $h_{d,i}$ is just our guess for the power consumption in trace $d$ for subkey $i$. We can get that easily using the `HW` array and `intermediate()` function we defined earlier:

```python
for bnum in range(0, 16):
    cpaoutput = [0]*256
    maxcpa = [0]*256
    for kguess in range(0, 256):
        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]
```

Now we can get $\bar{h_i}$:
```python
meanh = np.mean(hyp, dtype=np.float64)
```

and $\bar{t_j}$ is just the mean of all of our traces:
```python
meant = np.mean(traces, axis=0, dtype=np.float64)
```

Next, let's move on to calculating the whole sums using $h_{d,i}$ and $t_{d,j}$ and the values we just calculated:

```python
#For each trace, do the following
for tnum in range(numtraces):
    hdiff = (hyp[tnum] - meanh)
    tdiff = traces[tnum,:] - meant

    sumnum = sumnum + (hdiff*tdiff)
    sumden1 = sumden1 + hdiff*hdiff 
    sumden2 = sumden2 + tdiff*tdiff
```

We can now get the correlation for each of our subkey guesses, which we'll call `cpaoutput[]`:

```python
cpaoutput[kguess] = sumnum / np.sqrt( sumden1 * sumden2 )
```

We're almost done! All that's left is to use that correlation to figure out which subkey best matches our power traces. First off, we only care about absolute value of the correlation (that there is a linear correlation), not sign. Additionally, though this didn't factor into our correlation calculation, remember that each trace was actually made up of a bunch of sample points. This means that what we actually have is the correlation of each subkey guess to each sample point. Typically only a few points in the trace are correlating, and it's the maximum across the entire trace we are concerned with, so we can pick our correlation for each subkey by:

```python
maxcpa[kguess] = max(abs(cpaoutput[kguess]))
```

Finally, we can find the subkey that best matches our data by finding the one with the biggest correlation:

```python
bestguess[bnum] = np.argmax(maxcpa)
```

Putting it all together:

#### Finished Script

In [160]:
import numpy as np
from tqdm import tqdm

sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

HW = [bin(n).count("1") for n in range(0, 256)]

numtraces = np.shape(trace_array)[0] #total number of traces
numpoint = np.shape(trace_array)[1] #samples per trace

pt = textin_array
#knownkey = traces[0].key
cparefs = [0] * 16
bestguess = [0]*16

for bnum in tqdm(range(0, 16), desc='Attacking subkeys'):
    cpaoutput = [0] * 256
    maxcpa = [0] * 256
    for kguess in range(0, 256):

        # Initialize arrays &amp; variables to zero
        sumnum = np.zeros(numpoint)
        sumden1 = np.zeros(numpoint)
        sumden2 = np.zeros(numpoint)

        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]

        # Mean of hypothesis
        meanh = np.mean(hyp, dtype=np.float64)

        # Mean of all points in trace
        meant = np.mean(trace_array, axis=0, dtype=np.float64)

        # For each trace, do the following
        for tnum in range(0, numtraces):
            hdiff = (hyp[tnum] - meanh)
            tdiff = trace_array[tnum, :] - meant

            sumnum = sumnum + (hdiff * tdiff)
            sumden1 = sumden1 + hdiff * hdiff
            sumden2 = sumden2 + tdiff * tdiff

        cpaoutput[kguess] = sumnum / np.sqrt(sumden1 * sumden2)
        maxcpa[kguess] = max(abs(cpaoutput[kguess]))

    bestguess[bnum] = np.argmax(maxcpa)
    cparefs[bnum] = np.argsort(maxcpa)[::-1]

print("Best Key Guess: ", end="")
for b in bestguess: print("%02x " % b, end="")

Attacking subkeys: 100%|██████████| 16/16 [04:37<00:00, 17.35s/it]

Best Key Guess: c5 66 46 36 3d 46 68 05 b6 cc 20 dc 9b 0d 65 6e 